<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [2]:
!pip install sqlalchemy==1.3.9


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [3]:
!pip install ipython-sql

  Using cached SQLAlchemy-2.0.35-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
Using cached SQLAlchemy-2.0.35-cp311-cp311-win_amd64.whl (2.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [4]:
%load_ext sql

In [5]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [6]:
!pip install -q pandas

In [7]:
%sql sqlite:///my_data1.db

In [8]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [9]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [10]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [27]:
df = pd.read_sql_query("SELECT * FROM SPACEXTABLE", con)
print(df.columns)


unique_sites = df['Launch_Site'].unique()
print(unique_sites)

sql_query = """
SELECT "Launch_Site", COUNT(*) as LaunchCount
FROM SPACEXTABLE
GROUP BY "Launch_Site"
ORDER BY LaunchCount DESC
"""

launch_counts = pd.read_sql_query(sql_query, con)
print(launch_counts)



Index(['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
       'Landing_Outcome'],
      dtype='object')
['CCAFS LC-40' 'VAFB SLC-4E' 'KSC LC-39A' 'CCAFS SLC-40']
    Launch_Site  LaunchCount
0  CCAFS SLC-40           34
1   CCAFS LC-40           26
2    KSC LC-39A           25
3   VAFB SLC-4E           16



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [26]:
sql_query = """
SELECT *
FROM SPACEXTABLE
WHERE "Launch_Site" LIKE 'CCA%'
LIMIT 5
"""

five_cca = pd.read_sql_query(sql_query, con)
print(five_cca)

         Date Time (UTC) Booster_Version  Launch_Site  \
0  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                  0   
1  Dragon demo flight C1, two CubeSats, barrel of...                  0   
2                              Dragon demo flight C2                525   
3                                       SpaceX CRS-1                500   
4                                       SpaceX CRS-2                677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO (ISS)  NASA (COTS) NRO         Success  Failure (parachute)  

### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [28]:
sql_query = """
SELECT SUM(PAYLOAD_MASS__KG_) as TotalPayloadMass
FROM SPACEXTABLE
WHERE Customer = 'NASA (CRS)'
"""

result = pd.read_sql_query(sql_query, con)
print("Total payload mass carried by boosters launched by NASA (CRS):")
print(result['TotalPayloadMass'].values[0], "kg")

Total payload mass carried by boosters launched by NASA (CRS):
45596 kg


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [30]:
sql_query = """
SELECT AVG(PAYLOAD_MASS__KG_) as AveragePayloadMass
FROM SPACEXTABLE
WHERE "Booster_Version" = 'F9 v1.1'
"""

result = pd.read_sql_query(sql_query, con)
print("Average payload mass carried by boosters launched by NASA (CRS):")
print(result['AveragePayloadMass'].values[0], "kg")

Average payload mass carried by boosters launched by NASA (CRS):
2928.4 kg


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [33]:
sql_query = """
SELECT "Date", "Landing_Outcome"
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (ground pad)' 
ORDER BY Date ASC
LIMIT 1
"""

result = pd.read_sql_query(sql_query, con)


first_success_date = result['Date'].values[0]
print(f"The first successful landing on a ground pad was achieved on: {first_success_date}")
print(f"Landing Outcome: {result['Landing_Outcome'].values[0]}")


The first successful landing on a ground pad was achieved on: 2015-12-22
Landing Outcome: Success (ground pad)


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [38]:
sql_query = """
SELECT DISTINCT "Booster_Version"
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (drone ship)' 
    AND PAYLOAD_MASS__KG_ > 4000 
    AND PAYLOAD_MASS__KG_ < 6000
ORDER BY Booster_Version
"""

result = pd.read_sql_query(sql_query, con)

if not result.empty:
    print("Boosters with successful drone ship landings and payload mass between 4000 and 6000:")
    for booster in result['Booster_Version']:
        print(booster)
else:
    print("No boosters found meeting the specified criteria.")

Boosters with successful drone ship landings and payload mass between 4000 and 6000:
F9 FT  B1021.2
F9 FT  B1031.2
F9 FT B1022
F9 FT B1026


### Task 7




##### List the total number of successful and failure mission outcomes


In [39]:
sql_query = """
SELECT "Mission_Outcome", COUNT(*) as Count
FROM SPACEXTABLE
GROUP BY "Mission_Outcome"
ORDER BY Count DESC
"""

result = pd.read_sql_query(sql_query, con)

print("Mission Outcomes:")
print(result)

# Calculate total missions
total_missions = result['Count'].sum()
print(f"\nTotal number of missions: {total_missions}")


Mission Outcomes:
                    Mission_Outcome  Count
0                           Success     98
1  Success (payload status unclear)      1
2                          Success       1
3               Failure (in flight)      1

Total number of missions: 101


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [42]:
sql_query = """
SELECT DISTINCT "Booster_Version"
FROM SPACEXTABLE
WHERE "PAYLOAD_MASS__KG_" = (
    SELECT MAX(PAYLOAD_MASS__KG_)
    FROM SPACEXTABLE
)
"""

result = pd.read_sql_query(sql_query, con)

if not result.empty:
    print("Booster version(s) which have carried the maximum payload mass:")
    for booster in result['Booster_Version']:
        print(booster)
else:
    print("No booster versions found. This might indicate an issue with the data or query.")


Booster version(s) which have carried the maximum payload mass:
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2 
F9 B5 B1058.3 
F9 B5 B1051.6
F9 B5 B1060.3
F9 B5 B1049.7 


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [43]:
sql_query = """
SELECT 
    CASE 
        WHEN substr(Date, 6, 2) = '01' THEN 'January'
        WHEN substr(Date, 6, 2) = '02' THEN 'February'
        WHEN substr(Date, 6, 2) = '03' THEN 'March'
        WHEN substr(Date, 6, 2) = '04' THEN 'April'
        WHEN substr(Date, 6, 2) = '05' THEN 'May'
        WHEN substr(Date, 6, 2) = '06' THEN 'June'
        WHEN substr(Date, 6, 2) = '07' THEN 'July'
        WHEN substr(Date, 6, 2) = '08' THEN 'August'
        WHEN substr(Date, 6, 2) = '09' THEN 'September'
        WHEN substr(Date, 6, 2) = '10' THEN 'October'
        WHEN substr(Date, 6, 2) = '11' THEN 'November'
        WHEN substr(Date, 6, 2) = '12' THEN 'December'
    END AS Month,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM SPACEXTABLE
WHERE substr(Date, 0, 5) = '2015'
    AND Landing_Outcome != 'Failure (drone ship)'
ORDER BY substr(Date, 6, 2)
"""

result = pd.read_sql_query(sql_query, con)

if not result.empty:
    print("Records for failed drone ship landings in 2015:")
    print(result)
else:
    print("No records found matching the criteria.")

# Display the number of records
print(f"\nNumber of records: {len(result)}")

Records for failed drone ship landings in 2015:
      Month         Landing_Outcome Booster_Version  Launch_Site
0  February      Controlled (ocean)   F9 v1.1 B1013  CCAFS LC-40
1     March              No attempt   F9 v1.1 B1014  CCAFS LC-40
2     April              No attempt   F9 v1.1 B1016  CCAFS LC-40
3      June  Precluded (drone ship)   F9 v1.1 B1018  CCAFS LC-40
4  December    Success (ground pad)     F9 FT B1019  CCAFS LC-40

Number of records: 5


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [44]:
#'Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
#       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
#       'Landing_Outcome'

sql_query = """
SELECT 
    "Landing_Outcome",
    COUNT(*) as Count
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
    AND "Landing_Outcome" IS NOT NULL
    AND "Landing_Outcome" != ''
GROUP BY "Landing_Outcome"
ORDER BY Count DESC
"""

result = pd.read_sql_query(sql_query, con)

if not result.empty:
    print("Ranking of landing outcomes between 2010-06-04 and 2017-03-20:")
    for index, row in result.iterrows():
        print(f"Rank {index + 1}: {row['Landing_Outcome']} - Count: {row['Count']}")
else:
    print("No landing outcome data found for the specified date range.")

# Display the total number of missions in this period
total_missions = result['Count'].sum()
print(f"\nTotal number of missions in this period: {total_missions}")


Ranking of landing outcomes between 2010-06-04 and 2017-03-20:
Rank 1: No attempt - Count: 10
Rank 2: Success (drone ship) - Count: 5
Rank 3: Failure (drone ship) - Count: 5
Rank 4: Success (ground pad) - Count: 3
Rank 5: Controlled (ocean) - Count: 3
Rank 6: Uncontrolled (ocean) - Count: 2
Rank 7: Failure (parachute) - Count: 2
Rank 8: Precluded (drone ship) - Count: 1

Total number of missions in this period: 31


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
